### Import Libs

In [1]:
# Adding needed libraries and reading data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from databricks import koalas as ks
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Load Data

In [2]:
kdf_train = ks.read_csv('train.csv')
kdf_test =  ks.read_csv('test.csv')
print(kdf_train.shape)
print(kdf_test.shape)

(1460, 81)
(1459, 80)


### Drop Columns

In [3]:
train_labels = ks.sql('select Id, SalePrice from {kdf_train} ')
kdf_train.pop('SalePrice')
features = ks.concat([kdf_train, kdf_test])
features = features.drop(['Utilities', 'RoofMatl', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'Heating', 'LowQualFinSF',
               'BsmtFullBath', 'BsmtHalfBath', 'Functional', 'GarageYrBlt', 'GarageArea', 'GarageCond', 'WoodDeckSF',
               'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal'],
              axis=1)
print(features.shape)

(2919, 56)


### FillNa & Converting

In [15]:
# MSSubClass as str
features['MSSubClass'] = features['MSSubClass'].astype(str)

# MSZoning NA in pred. filling with most popular values
features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])

# LotFrontage  NA in all. I suppose NA means 0
features['LotFrontage'] = features['LotFrontage'].fillna(features['LotFrontage'].mean())

# Alley  NA in all. NA means no access
features['Alley'] = features['Alley'].fillna('NOACCESS')

# Converting OverallCond to str
features.OverallCond = features.OverallCond.astype(str)

# MasVnrType NA in all. filling with most popular values
features['MasVnrType'] = features['MasVnrType'].fillna(features['MasVnrType'].mode()[0])

# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
# NA in all. NA means No basement
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('NoBSMT')

# TotalBsmtSF  NA in pred. I suppose NA means 0
features['TotalBsmtSF'] = features['TotalBsmtSF'].fillna(0)

# Electrical NA in pred. filling with most popular values
features['Electrical'] = features['Electrical'].fillna(features['Electrical'].mode()[0])

# KitchenAbvGr to categorical
features['KitchenAbvGr'] = features['KitchenAbvGr'].astype(str)

# KitchenQual NA in pred. filling with most popular values
features['KitchenQual'] = features['KitchenQual'].fillna(features['KitchenQual'].mode()[0])

# FireplaceQu  NA in all. NA means No Fireplace
features['FireplaceQu'] = features['FireplaceQu'].fillna('NoFP')

# GarageType, GarageFinish, GarageQual  NA in all. NA means No Garage
for col in ('GarageType', 'GarageFinish', 'GarageQual'):
    features[col] = features[col].fillna('NoGRG')

# GarageCars  NA in pred. I suppose NA means 0
features['GarageCars'] = features['GarageCars'].fillna(0.0)

# SaleType NA in pred. filling with most popular values
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

# Year and Month to categorical
features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

# Adding total sqfootage feature and removing Basement, 1st and 2nd floor features
features['TotalSF'] = features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
features = features.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1)

TypeError: string addition can only be applied to string series or literals.

### Log transformation

In [7]:
## Standardizing numeric features
numeric_features = features.loc[:,['LotFrontage', 'LotArea', 'GrLivArea', 'TotalSF']]
#numeric_features_standardized = (numeric_features - numeric_features.mean())/numeric_features.std()

KeyError: "['TotalSF'] not in index"

### Converting categorical data to dummies

In [16]:
print(features.shape)

features = ks.get_dummies(data=features, columns=["Condition1"])
print(features.shape)
print(features.columns)
features
# Getting Dummies from Condition1 and Condition2
# conditions = set([x for x in features['Condition1']] + [x for x in features['Condition2']])
# dummies = ks.DataFrame(data=np.zeros((len(features.index), len(conditions))),
#                        index=features.index, columns=conditions)
# for i, cond in enumerate(zip(features['Condition1'], features['Condition2'])):
#     dummies.ix[i, cond] = 1
# features = ks.concat([features, dummies.add_prefix('Condition_')], axis=1)
# features = features.drop(['Condition1', 'Condition2'], axis=1)

# # Getting Dummies from Exterior1st and Exterior2nd
# exteriors = set([x for x in features['Exterior1st']] + [x for x in features['Exterior2nd']])
# dummies = ks.DataFrame(data=np.zeros((len(features.index), len(exteriors))),
#                        index=features.index, columns=exteriors)
# for i, ext in enumerate(zip(features['Exterior1st'], features['Exterior2nd'])):
#     dummies.ix[i, ext] = 1
# features = ks.concat([features, dummies.add_prefix('Exterior_')], axis=1)
# features = features.drop(['Exterior1st', 'Exterior2nd', 'Exterior_nan'], axis=1)

# # Getting Dummies from all other categorical vars
# for col in features.dtypes[features.dtypes == 'object'].index:
#     for_dummy = features.pop(col)
#     features = pd.concat([features, pd.get_dummies(for_dummy, prefix=col)], axis=1)

(2919, 64)
(2919, 64)
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'TotalBsmtSF', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageQual',
       'PavedDrive', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition',
       'Condition1_Artery', 'Condition1_Feedr', 'Condition1_Norm',
       'Condition1_PosA', 'Condition1_PosN', 'Condition1_RRAe',
       'Condition1_RRAn', 

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageQual,PavedDrive,MoSold,YrSold,SaleType,SaleCondition,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,Inside,Gtl,CollgCr,Norm,1Fam,2Story,7,5,2003,2003,Gable,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,856,Ex,Y,SBrkr,856,854,1710,2,1,3,1,Gd,8,0,NA,Attchd,RFn,2,TA,Y,2,2008,WD,Normal,0,0,1,0,0,0,0,0,0
1,2,20,RL,80,9600,Pave,NA,Reg,Lvl,FR2,Gtl,Veenker,Norm,1Fam,1Story,6,8,1976,1976,Gable,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,1262,Ex,Y,SBrkr,1262,0,1262,2,0,3,1,TA,6,1,TA,Attchd,RFn,2,TA,Y,5,2007,WD,Normal,0,1,0,0,0,0,0,0,0
2,3,60,RL,68,11250,Pave,NA,IR1,Lvl,Inside,Gtl,CollgCr,Norm,1Fam,2Story,7,5,2001,2002,Gable,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,920,Ex,Y,SBrkr,920,866,1786,2,1,3,1,Gd,6,1,TA,Attchd,RFn,2,TA,Y,9,2008,WD,Normal,0,0,1,0,0,0,0,0,0
3,4,70,RL,60,9550,Pave,NA,IR1,Lvl,Corner,Gtl,Crawfor,Norm,1Fam,2Story,7,5,1915,1970,Gable,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,756,Gd,Y,SBrkr,961,756,1717,1,0,3,1,Gd,7,1,Gd,Detchd,Unf,3,TA,Y,2,2006,WD,Abnorml,0,0,1,0,0,0,0,0,0
4,5,60,RL,84,14260,Pave,NA,IR1,Lvl,FR2,Gtl,NoRidge,Norm,1Fam,2Story,8,5,2000,2000,Gable,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,1145,Ex,Y,SBrkr,1145,1053,2198,2,1,4,1,Gd,9,1,TA,Attchd,RFn,3,TA,Y,12,2008,WD,Normal,0,0,1,0,0,0,0,0,0
5,6,50,RL,85,14115,Pave,NA,IR1,Lvl,Inside,Gtl,Mitchel,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,VinylSd,VinylSd,None,TA,TA,Wood,Gd,TA,No,GLQ,Unf,796,Ex,Y,SBrkr,796,566,1362,1,1,1,1,TA,5,0,NA,Attchd,Unf,2,TA,Y,10,2009,WD,Normal,0,0,1,0,0,0,0,0,0
6,7,20,RL,75,10084,Pave,NA,Reg,Lvl,Inside,Gtl,Somerst,Norm,1Fam,1Story,8,5,2004,2005,Gable,VinylSd,VinylSd,Stone,Gd,TA,PConc,Ex,TA,Av,GLQ,Unf,1686,Ex,Y,SBrkr,1694,0,1694,2,0,3,1,Gd,7,1,Gd,Attchd,RFn,2,TA,Y,8,2007,WD,Normal,0,0,1,0,0,0,0,0,0
7,8,60,RL,NA,10382,Pave,NA,IR1,Lvl,Corner,Gtl,NWAmes,Norm,1Fam,2Story,7,6,1973,1973,Gable,HdBoard,HdBoard,Stone,TA,TA,CBlock,Gd,TA,Mn,ALQ,BLQ,1107,Ex,Y,SBrkr,1107,983,2090,2,1,3,1,TA,7,2,TA,Attchd,RFn,2,TA,Y,11,2009,WD,Normal,0,0,0,0,1,0,0,0,0
8,9,50,RM,51,6120,Pave,NA,Reg,Lvl,Inside,Gtl,OldTown,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,BrkFace,Wd Shng,None,TA,TA,BrkTil,TA,TA,No,Unf,Unf,952,Gd,Y,FuseF,1022,752,1774,2,0,2,2,TA,8,2,TA,Detchd,Unf,2,Fa,Y,4,2008,WD,Abnorml,1,0,0,0,0,0,0,0,0
9,10,190,RL,50,7420,Pave,NA,Reg,Lvl,Corner,Gtl,BrkSide,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,MetalSd,MetalSd,None,TA,TA,BrkTil,TA,TA,No,GLQ,Unf,991,Ex,Y,SBrkr,1077,0,1077,1,0,2,2,TA,5,2,TA,Attchd,RFn,1,Gd,Y,1,2008,WD,Normal,1,0,0,0,0,0,0,0,0


### Plot

In [ ]:
# features.plot.hist('OverallCond	')
# type(features)

### Split Train & test df

In [17]:
print(features.shape)
kdf_train = features.iloc[:1460,:]
print(type(train_labels) )
#train_labels = train_labels.to_frame()
print(type(train_labels) )
#print(train_labels)




(2919, 64)
databricks.koalas.frame.DataFrame
databricks.koalas.frame.DataFrame


### Merge train with lable

In [11]:
kdf_test = ks.sql('select * from {features} where Id > 1460')
kdf_train = kdf_train.merge(train_labels, left_on='Id', right_on='Id' )
print(type(train_labels) )
print(kdf_train.shape)
print(kdf_test.shape)

databricks.koalas.frame.DataFrame
(1460, 65)
(1459, 64)


### Gradient Boosting

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [13]:
sprkdf_train = kdf_train.to_spark().select('OverallQual', 'SalePrice')
vectorAssembler = VectorAssembler(inputCols = ['OverallQual'], outputCol = 'feature_vector')
vhouse_df = vectorAssembler.transform(sprkdf_train)
vhouse_df = vhouse_df.select(['feature_vector', 'SalePrice'])
vhouse_df.show(3)



# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = vhouse_df.randomSplit([0.7, 0.3])


# Train a GBT model.
gbt = GBTRegressor(featuresCol = 'feature_vector', labelCol='SalePrice')
gbt_model = gbt.fit(trainingData)

gbt_predictions = gbt_model.transform(testData)
gbt_predictions.select('prediction', 'SalePrice', 'feature_vector').show(5)

+--------------+---------+
|feature_vector|SalePrice|
+--------------+---------+
|         [7.0]|   208500|
|         [6.0]|   181500|
|         [7.0]|   223500|
+--------------+---------+
only showing top 3 rows

+------------------+---------+--------------+
|        prediction|SalePrice|feature_vector|
+------------------+---------+--------------+
|48096.742275171455|    60000|         [2.0]|
| 86543.35214047466|    52000|         [3.0]|
| 86543.35214047466|    58500|         [3.0]|
| 86543.35214047466|    67000|         [3.0]|
| 86543.35214047466|    92900|         [3.0]|
+------------------+---------+--------------+
only showing top 5 rows



In [14]:
gbt_evaluator = RegressionEvaluator(
    labelCol="SalePrice", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 50320.4
